In [8]:
import requests
import lxml
import bs4
import json
import pandas as pd
from time import sleep

In [9]:
class Search_Engine_Comparison():
    def __init__(self,url):
        self.queries = []
        self.url = url 
        self.result = {}
        self.json_inp = {}
        self.ref_json = {}
        self.matches = {}
        self.df = None
        
    def extract_input_queries(self):
        with open('100QueriesSet2.txt',mode='r') as fopen:
            self.queries = fopen.readlines()
        self.queries = list(map(lambda q: q[0:len(q)-2],self.queries))
    
    def connect_to_bing_engine(self,queries):
        for query in queries:
            q = query.replace(" ","+")
            q = q[0:len(q)]
            full_url = self.url + q
            req = requests.get(full_url)
            soup = bs4.BeautifulSoup(req.text,"lxml")
            content = soup.select(".b_algo")
            if len(content) < 10:
                full_url = full_url + "&count=30"
                new_req = requests.get(full_url)
                new_soup = bs4.BeautifulSoup(new_req.text,"lxml")
                new_content = new_soup.select(".b_algo")
                self.result[query] = new_content
            else:
                self.result[query] = content
            sleep(5)
            
    def connect_to_yahoo_engine(self,queries):
        for query in queries:
            q = query.replace(" ","+")
            q = q[0:len(q)]
            i = 1
            content = []
            count = 0
            while len(content) < 10:
                if count == 5:
                    break
                full_url = self.url + q + '&b=' + str(i)
                i += 1
                req = requests.get(full_url)
                soup = bs4.BeautifulSoup(req.text,"lxml")
                content.extend(soup.find_all("a",class_='d-ib fz-20 lh-26 td-hu tc va-bot mxw-100p'))
                count += 1
            self.result[query] = content
            sleep(2)
    
    def process_bing_content(self):
        for key,value in self.result.items():
            links = []
            for item in value:
                links.append(item.select('a')[0]['href'])
            self.json_inp[key] = links
            
    def process_yahoo_content(self):
        for key,value in self.result.items():
            links = []
            for item in value:
                links.append(item['href'])
            self.json_inp[key] = links
            
    def generate_json_file(self):
        with open("hw1.json",mode="w") as fopen:
            json.dump(self.json_inp,fopen,indent=4)
            
    def extract_reference_search_result(self):
        with open('Google_Result2.json',mode='r') as fopen:
            self.ref_json = json.load(fopen)
            
    def generate_ranks(self):
        # dictionary key = reference rank
        # dictionary value = search result rank
        for key,value in self.json_inp.items():
            ref_res = self.ref_json.get(key)
            if ref_res != None:
                rank = []
                for i in range(0,len(value)):
                    pattern = value[i].replace('www.','')
                    if pattern[0:5] == 'https':
                        pattern = pattern[5:]
                    elif pattern[0:4] == 'http':
                        pattern = pattern[4:]
                    if pattern[len(pattern)-1] == '/':
                        pattern = pattern[0:len(pattern)-1]
                    for j in range(0,len(ref_res)):
                        if pattern in ref_res[j]:
                            search_rank = i+1
                            ref_rank = j+1
                            rank.append((ref_rank,search_rank))
                self.matches[key] = rank
    
    def calculate_score(self):
        df_dat = []
        avg_n = 0
        avg_perc = 0
        avg_coeff = 0
        for key,value in self.matches.items():
            index = self.queries.index(key)+1
            n = len(value)
            if n > 0:
                avg_n += n
                overlap_perc = (len(value)/10)*100
                avg_perc += overlap_perc
                if n == 1:
                    ref,srch = value[0]
                    if ref == srch:
                        rho = 1
                    else:
                        rho = 0
                else:
                    n = len(value)
                    dist_sum = 0
                    for ref,srch in value:
                        dist_sum += (ref-srch)**2
                    rho = 1 - ((6*dist_sum)/(n*((n**2)-1)))
                avg_coeff += rho
                df_dat.append(['Query '+str(index),n,overlap_perc,rho])
        size = len(df_dat)
        df_dat.append(['Averages',avg_n/size,avg_perc/size,avg_coeff/size])
        self.df = pd.DataFrame(df_dat,columns=['Queries','Number of Overlapping Results','Percent Overlap','Spearman Coefficient'])
        
    def generate_csv_file(self):
        self.df.to_csv('hw1.csv',index=False)
        
    def start_comparison(self):
        self.extract_input_queries()
        self.extract_reference_search_result()
        engine = input("enter the search engine to be used: ")
        if engine.lower() == 'bing':
            i=0
            while i < 100:
                self.connect_to_bing_engine(self.queries[i:i+10])
                i = i+10
            self.process_bing_content()
        elif engine.lower() == 'yahoo':
            self.connect_to_yahoo_engine(self.queries)
            self.process_yahoo_content()
        self.generate_json_file()
        self.generate_ranks()
        self.calculate_score()
        self.generate_csv_file()

In [10]:
yahoo = Search_Engine_Comparison('https://search.yahoo.com/search?p=')

In [11]:
yahoo.start_comparison()

enter the search engine to be used: yahoo
